In [456]:
import pandas as pd
import numpy as np
import time

import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')
from sklearn import preprocessing


# Converting Json data to DataFrame

In [457]:
data = pd.read_json('computers_train_xlarge.json',lines = True)
data.to_csv('train.csv',index = False)
df = pd.read_csv('train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68461 entries, 0 to 68460
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id_left                 68461 non-null  int64 
 1   category_left           68461 non-null  object
 2   cluster_id_left         68461 non-null  int64 
 3   id_right                68461 non-null  int64 
 4   category_right          68461 non-null  object
 5   cluster_id_right        68461 non-null  int64 
 6   label                   68461 non-null  int64 
 7   pair_id                 68461 non-null  object
 8   brand_left              34233 non-null  object
 9   brand_right             34245 non-null  object
 10  description_left        47460 non-null  object
 11  description_right       48360 non-null  object
 12  keyValuePairs_left      18765 non-null  object
 13  keyValuePairs_right     20275 non-null  object
 14  price_left              11521 non-null  object
 15  pr

In [458]:
df

,id_left,category_left,cluster_id_left,id_right,category_right,cluster_id_right,label,pair_id,brand_left,brand_right,description_left,description_right,keyValuePairs_left,keyValuePairs_right,price_left,price_right,specTableContent_left,specTableContent_right,title_left,title_right
0,2551242,Computers_and_Accessories,79457,16272671,Computers_and_Accessories,79457,1,2551242#16272671,"""Corsair""@en",NaN,"""DDR4, 2666MHz, CL16, 1.2v, XMP 2.0, Lifetime ...",NaN,"{'Memory Type': 'DDR4 (PC4-21300)', 'Capacity'...","{'Categorie': 'Geheugen intern', 'Merk': 'Cors...",NaN,NaN,Memory Type DDR4 (PC4-21300) Capacity 64GB (4...,Categorie Geheugen intern Merk Corsair Produc...,"""Corsair Vengeance LPX Black 64GB (4x16GB) DD...","""Corsair Vengeance LPX CMK64GX4M4A2666C16 - P..."
1,16757469,Computers_and_Accessories,63553,16476204,Computers_and_Accessories,63553,1,16757469#16476204,"""HP Enterprise""","""HP Enterprise""","""Description:2 x 72GB 2.5-inch Serial Attached...","""Description:10 x 72GB 2.5-inch Serial Attach...","{'Category': 'Proliant HardDrive', 'Sub-Catego...","{'Category': 'Proliant HardDrive', 'Sub-Catego...",NaN,"""CAD"", ""$757.74 CAD""",Specifications: Category Proliant HardDrive S...,Specifications: Category Proliant HardDrive S...,"""DH0072BALWL HP 72-GB 3G 15K 2.5 DP SAS"", ""Nu...","""DH0072BALWL HP 72-GB 3G 15K 2.5 DP SAS"" ""Null"""
2,232007,Computers_and_Accessories,212917,16442945,Computers_and_Accessories,14319765,0,232007#16442945,"""SanDisk""","""HP Enterprise""","""SDSDJ-1024 BXP 1GB 9p SD Class 2 Secure Digi...","""Description:Genuine HPE 1GB FBD PC2-5300(2x5...",NaN,"{'Category': 'Proliant Memory', 'Sub-Category'...","""USD"", ""7.98""","""USD"", ""$69.95""",NaN,Specifications: Category Proliant Memory Sub-...,"""SanDisk SDSDJ-1024 BXP 1GB 9p SD Class 2 Sec...","""397409-B21 HP 1GB (2x512MB) PC2-5300 SDRAM"" ..."
3,2066119,Computers_and_Accessories,667772,12411100,Computers_and_Accessories,667772,1,2066119#12411100,NaN,"""KINGSTON""","""DISCO DURO INTERNO SOLIDO HDD SSD""@es",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""DISCO DURO INTERNO SOLIDO HDD SSD KINGSTON V...","""DISCO DURO SSD Kingston Technology SSDNow V3..."
4,6656540,Computers_and_Accessories,505997,2639431,Computers_and_Accessories,154118,0,6656540#2639431,NaN,"""Corsair""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Corsair Vengeance LED 32GB (2 x 16GB) DDR4 D...","""Corsair - Vengeance LPX 32GB (2 x 16GB) DDR4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68456,6493497,Computers_and_Accessories,222681,16149764,Computers_and_Accessories,502396,0,6493497#16149764,"""HP Enterprise""","""HP Enterprise""","""Description:5 x 300GB 2.5-inch Serial Attache...","""Description:2 x 72GB 2.5-inch Serial Attache...","{'Category': 'Proliant HardDrive', 'Sub-Catego...","{'Category': 'Proliant HardDrive', 'Sub-Catego...",NaN,"""$151.40"", ""USD""",Specifications: Category Proliant HardDrive S...,Specifications: Category Proliant HardDrive S...,"""DG0300FARVV HP 300-GB 6G 10K 2.5 DP SAS"", ""N...","""Null"" ""512743-001 HP 72-GB 6G 15K 2.5 DP SAS"""
68457,17075265,Computers_and_Accessories,944407,17346839,Computers_and_Accessories,825203,0,17075265#17346839,"""Samsung""",NaN,NaN,"""Cost-effective SSD featuring TurboWrite and ...",NaN,NaN,NaN,NaN,NaN,NaN,"""Samsung - 840 EVO 250GB 2.5"" Solid State Dri...","""SSD 750 EVO 2.5” SATA III 120GB ""@en"
68458,16408794,Computers_and_Accessories,144052,3675781,Computers_and_Accessories,51801,0,16408794#3675781,"""INTEL""@fr","""Intel""@en",NaN,"""Quad Core Technology, 3.6GHz clock speed, 8MB...",NaN,"{'Processor Number': 'I7 7700', 'Number of Cor...",NaN,NaN,NaN,Processor Number I7 7700 Number of Cores 4 Nu...,"""Socket H4 1151 - Coffee Lake Core i7-8700K 6...","""7th Generation Intel® Core™ i7 7700 3.6GHz S..."
68459,13925964,Computers_and_Accessories,43816,15659664,Computers_and_Accessories,478188,0,13925964#15659664,NaN,NaN,"""Built to WD's highest standards of quality an...",NaN,NaN,NaN,"""$"", "

# Data Cleaning : Removing Unwanted Columns

In [459]:
df.drop(df.columns[[2,5,8,9,12,13,14,15,16,17]],axis=1,inplace =True)
df

,id_left,category_left,id_right,category_right,label,pair_id,description_left,description_right,title_left,title_right
0,2551242,Computers_and_Accessories,16272671,Computers_and_Accessories,1,2551242#16272671,"""DDR4, 2666MHz, CL16, 1.2v, XMP 2.0, Lifetime ...",NaN,"""Corsair Vengeance LPX Black 64GB (4x16GB) DD...","""Corsair Vengeance LPX CMK64GX4M4A2666C16 - P..."
1,16757469,Computers_and_Accessories,16476204,Computers_and_Accessories,1,16757469#16476204,"""Description:2 x 72GB 2.5-inch Serial Attached...","""Description:10 x 72GB 2.5-inch Serial Attach...","""DH0072BALWL HP 72-GB 3G 15K 2.5 DP SAS"", ""Nu...","""DH0072BALWL HP 72-GB 3G 15K 2.5 DP SAS"" ""Null"""
2,232007,Computers_and_Accessories,16442945,Computers_and_Accessories,0,232007#16442945,"""SDSDJ-1024 BXP 1GB 9p SD Class 2 Secure Digi...","""Description:Genuine HPE 1GB FBD PC2-5300(2x5...","""SanDisk SDSDJ-1024 BXP 1GB 9p SD Class 2 Sec...","""397409-B21 HP 1GB (2x512MB) PC2-5300 SDRAM"" ..."
3,2066119,Computers_and_Accessories,12411100,Computers_and_Accessories,1,2066119#12411100,"""DISCO DURO INTERNO SOLIDO HDD SSD""@es",NaN,"""DISCO DURO INTERNO SOLIDO HDD SSD KINGSTON V...","""DISCO DURO SSD Kingston Technology SSDNow V3..."
4,6656540,Computers_and_Accessories,2639431,Computers_and_Accessories,0,6656540#2639431,NaN,NaN,"""Corsair Vengeance LED 32GB (2 x 16GB) DDR4 D...","""Corsair - Vengeance LPX 32GB (2 x 16GB) DDR4..."
...,...,...,...,...,...,...,...,...,...,...
68456,6493497,Computers_and_Accessories,16149764,Computers_and_Accessories,0,6493497#16149764,"""Description:5 x 300GB 2.5-inch Serial Attache...","""Description:2 x 72GB 2.5-inch Serial Attache...","""DG0300FARVV HP 300-GB 6G 10K 2.5 DP SAS"", ""N...","""Null"" ""512743-001 HP 72-GB 6G 15K 2.5 DP SAS"""
68457,17075265,Computers_and_Accessories,17346839,Computers_and_Accessories,0,17075265#17346839,NaN,"""Cost-effective SSD featuring TurboWrite and ...","""Samsung - 840 EVO 250GB 2.5"" Solid State Dri...","""SSD 750 EVO 2.5” SATA III 120GB ""@en"
68458,16408794,Computers_and_Accessories,3675781,Computers_and_Accessories,0,16408794#3675781,NaN,"""Quad Core Technology, 3.6GHz clock speed, 8MB...","""Socket H4 1151 - Coffee Lake Core i7-8700K 6...","""7th Generation Intel® Core™ i7 7700 3.6GHz S..."
68459,13925964,Computers_and_Accessories,15659664,Computers_and_Accessories,0,13925964#15659664,"""Built to WD's highest standards of quality an...",NaN,"""WD Blue WD5000AZLX - hard drive 500 GB SATA ...","""m rock ships""@en-US ""M-ROCK Ships New Camera..."


# Data Analysis 

In [460]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68461 entries, 0 to 68460
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id_left            68461 non-null  int64 
 1   category_left      68461 non-null  object
 2   id_right           68461 non-null  int64 
 3   category_right     68461 non-null  object
 4   label              68461 non-null  int64 
 5   pair_id            68461 non-null  object
 6   description_left   47460 non-null  object
 7   description_right  48360 non-null  object
 8   title_left         68461 non-null  object
 9   title_right        68461 non-null  object
dtypes: int64(3), object(7)
memory usage: 5.2+ MB


In [461]:
df['label'].value_counts()

0    58771
1     9690
Name: label, dtype: int64

In [462]:
df['category_right'].value_counts()


Computers_and_Accessories     66972
Other_Electronics               463
Video_Games                     405
Office_Products                 262
Luggage_and_Travel_Gear         122
Cellphones_and_Accessories      109
Camera_and_Photo                 74
Musical_Instruments              44
Tools_and_Home_Improvement       10
Name: category_right, dtype: int64

In [463]:
df['id_left'].value_counts()


14619165    43
8364399     42
5411511     41
8167053     40
4927241     40
            ..
6810432      1
16476204     1
10969778     1
5872903      1
4604957      1
Name: id_left, Length: 4287, dtype: int64

In [464]:
(df['id_left'] < 0).sum()

0

In [465]:
df['id_right'].value_counts()

7900893     116
12050629    107
7440444     106
6443065      97
13889537     97
           ... 
5199342       1
16567684      1
16123988      1
13661066      1
7867351       1
Name: id_right, Length: 4144, dtype: int64

In [466]:
df['description_left'].isnull().sum()

21001

In [467]:
df['title_left'].isnull().sum()

0

# Filling the Null values of Description Columns by Title

In [468]:
df.description_left.fillna(df.title_left, inplace = True)
df['title_left']

0         "Corsair Vengeance LPX Black 64GB (4x16GB) DD...
1         "DH0072BALWL HP 72-GB 3G 15K 2.5 DP SAS", "Nu...
2         "SanDisk SDSDJ-1024 BXP 1GB 9p SD Class 2 Sec...
3         "DISCO DURO INTERNO SOLIDO HDD SSD KINGSTON V...
4         "Corsair Vengeance LED 32GB (2 x 16GB) DDR4 D...
                               ...                        
68456     "DG0300FARVV HP 300-GB 6G 10K 2.5 DP SAS", "N...
68457     "Samsung - 840 EVO 250GB 2.5" Solid State Dri...
68458     "Socket H4 1151 - Coffee Lake Core i7-8700K 6...
68459     "WD Blue WD5000AZLX - hard drive 500 GB SATA ...
68460     "Acer KA KA240H 24" Full HD TN Negro pantalla...
Name: title_left, Length: 68461, dtype: object

In [469]:
df['description_left'].isnull().sum()

0

In [470]:
df['description_right'].isnull().sum()

20101

In [471]:
df['title_right'].isnull().sum()

0

In [472]:
df.description_right.fillna(df.title_left, inplace = True)
df['description_right']

0         "Corsair Vengeance LPX Black 64GB (4x16GB) DD...
1         "Description:10 x 72GB 2.5-inch Serial Attach...
2         "Description:Genuine HPE 1GB FBD PC2-5300(2x5...
3         "DISCO DURO INTERNO SOLIDO HDD SSD KINGSTON V...
4         "Corsair Vengeance LED 32GB (2 x 16GB) DDR4 D...
                               ...                        
68456     "Description:2 x 72GB 2.5-inch Serial Attache...
68457     "Cost-effective SSD featuring TurboWrite and ...
68458    "Quad Core Technology, 3.6GHz clock speed, 8MB...
68459     "WD Blue WD5000AZLX - hard drive 500 GB SATA ...
68460    "Longitud diagonal: 24 "; Tamaño: 16:9; Tecnol...
Name: description_right, Length: 68461, dtype: object

In [473]:
df['description_right'].isnull().sum()

0

# Comparing category left and category right

In [474]:
df['category_match'] = np.where(df['category_left'] == df['category_right'],1,0)
df.head()

#1-Match / 0-Unmatch


,id_left,category_left,id_right,category_right,label,pair_id,description_left,description_right,title_left,title_right,category_match
0,2551242,Computers_and_Accessories,16272671,Computers_and_Accessories,1,2551242#16272671,"""DDR4, 2666MHz, CL16, 1.2v, XMP 2.0, Lifetime ...","""Corsair Vengeance LPX Black 64GB (4x16GB) DD...","""Corsair Vengeance LPX Black 64GB (4x16GB) DD...","""Corsair Vengeance LPX CMK64GX4M4A2666C16 - P...",1
1,16757469,Computers_and_Accessories,16476204,Computers_and_Accessories,1,16757469#16476204,"""Description:2 x 72GB 2.5-inch Serial Attached...","""Description:10 x 72GB 2.5-inch Serial Attach...","""DH0072BALWL HP 72-GB 3G 15K 2.5 DP SAS"", ""Nu...","""DH0072BALWL HP 72-GB 3G 15K 2.5 DP SAS"" ""Null""",1
2,232007,Computers_and_Accessories,16442945,Computers_and_Accessories,0,232007#16442945,"""SDSDJ-1024 BXP 1GB 9p SD Class 2 Secure Digi...","""Description:Genuine HPE 1GB FBD PC2-5300(2x5...","""SanDisk SDSDJ-1024 BXP 1GB 9p SD Class 2 Sec...","""397409-B21 HP 1GB (2x512MB) PC2-5300 SDRAM"" ...",1
3,2066119,Computers_and_Accessories,12411100,Computers_and_Accessories,1,2066119#12411100,"""DISCO DURO INTERNO SOLIDO HDD SSD""@es","""DISCO DURO INTERNO SOLIDO HDD SSD KINGSTON V...","""DISCO DURO INTERNO SOLIDO HDD SSD KINGSTON V...","""DISCO DURO SSD Kingston Technology SSDNow V3...",1
4,6656540,Computers_and_Accessories,2639431,Computers_and_Accessories,0,6656540#2639431,"""Corsair Vengeance LED 32GB (2 x 16GB) DDR4 D...","""Corsair Vengeance LED 32GB (2 x 16GB) DDR4 D...","""Corsair Vengeance LED 32GB (2 x 16GB) DDR4 D...","""Corsair - Vengeance LPX 32GB (2 x 16GB) DDR4...",1


In [475]:
df['category_match'].value_counts()


1    65018
0     3443
Name: category_match, dtype: int64

In [476]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68461 entries, 0 to 68460
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id_left            68461 non-null  int64 
 1   category_left      68461 non-null  object
 2   id_right           68461 non-null  int64 
 3   category_right     68461 non-null  object
 4   label              68461 non-null  int64 
 5   pair_id            68461 non-null  object
 6   description_left   68461 non-null  object
 7   description_right  68461 non-null  object
 8   title_left         68461 non-null  object
 9   title_right        68461 non-null  object
 10  category_match     68461 non-null  int32 
dtypes: int32(1), int64(3), object(7)
memory usage: 5.5+ MB


In [477]:
df.drop(df.index[df['category_match'] == 0],inplace = True)

In [478]:
df.drop(df.index[df['label'] == 0],inplace = True)

In [479]:
df['label'].value_counts()

1    9690
Name: label, dtype: int64

In [480]:
df['category_match'].value_counts()

1    9690
Name: category_match, dtype: int64

In [481]:
newdf=df.copy()
newdf.head()

,id_left,category_left,id_right,category_right,label,pair_id,description_left,description_right,title_left,title_right,category_match
0,2551242,Computers_and_Accessories,16272671,Computers_and_Accessories,1,2551242#16272671,"""DDR4, 2666MHz, CL16, 1.2v, XMP 2.0, Lifetime ...","""Corsair Vengeance LPX Black 64GB (4x16GB) DD...","""Corsair Vengeance LPX Black 64GB (4x16GB) DD...","""Corsair Vengeance LPX CMK64GX4M4A2666C16 - P...",1
1,16757469,Computers_and_Accessories,16476204,Computers_and_Accessories,1,16757469#16476204,"""Description:2 x 72GB 2.5-inch Serial Attached...","""Description:10 x 72GB 2.5-inch Serial Attach...","""DH0072BALWL HP 72-GB 3G 15K 2.5 DP SAS"", ""Nu...","""DH0072BALWL HP 72-GB 3G 15K 2.5 DP SAS"" ""Null""",1
3,2066119,Computers_and_Accessories,12411100,Computers_and_Accessories,1,2066119#12411100,"""DISCO DURO INTERNO SOLIDO HDD SSD""@es","""DISCO DURO INTERNO SOLIDO HDD SSD KINGSTON V...","""DISCO DURO INTERNO SOLIDO HDD SSD KINGSTON V...","""DISCO DURO SSD Kingston Technology SSDNow V3...",1
7,7759463,Computers_and_Accessories,17348137,Computers_and_Accessories,1,7759463#17348137,up to 2.5 GHz burst frequency connect and sha...,""" Be free to create, connect and share in more...","""HP Pavilion x360 - 11-ad018ca""@en","""HP Pavilion x360 - 11-ad018ca""@en 11-ad018ca...",1
8,9650771,Computers_and_Accessories,11456656,Computers_and_Accessories,1,9650771#11456656,"""Startech 1 Port PCI 10/100/1000 32 Bit Gigab...","""Startech 1 Port PCI 10/100/1000 32 Bit Gigab...","""Startech 1 Port PCI 10/100/1000 32 Bit Gigab...","""StarTech.com 10 100 1000Mbps 32 bit PCI Gigi...",1


# Applying Maching Numbers function to match the products Features 

In [482]:
def matching_numbers(title_right, title_left):

    title_right = set(re.findall(r'[0-9]+', description_right))
    title_left = set(re.findall(r'[0-9]+', title_left))    
    union = title_right.union(title_left)
    intersection = title_right.intersection(title_left)

    if len(title_right)==0 and len(title_left) == 0:
        return 1
    else:
        return (len(intersection)/ len(union))


In [483]:
def remove_spaces(text):
    text=text.strip()
    text=text.split()
    return ' '.join(text)

contraction = {'cause':'because',
              'aint': 'am not',
              'aren\'t': 'are not'}

def mapping_replacer(x,dic):
    for words in dic.keys():
        if ' ' + words + ' ' in x:
            x=x.replace(' '+ words +' ' ,' '+dic[words]+' ' )
    return x



In [484]:
#Stemming, lemmetisation and tokenisation
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer

nltk.LancasterStemmer
ls = LancasterStemmer()
lem = WordNetLemmatizer()
def lexicon_normalization(text):
    words = word_tokenize(text) 
    
    
    # 1- Stemming
    words_stem = [ls.stem(w) for w in words]
    
    # 2- Lemmatization
    words_lem = [lem.lemmatize(w) for w in words_stem]
    return words_lem



In [485]:

#Handling stopwords
from collections import Counter
def remove_stopword(text):
    stop_words = stopwords.words('english')
    stopwords_dict = Counter(stop_words)
    text = ' '.join([word for word in text.split() if word not in stopwords_dict])
    return text


In [486]:


#Removing links, brackets, numbers, punctuations etc.

def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', ' ', text)
    text = re.sub('https?://\S+|www\.\S+', ' ', text)
    text = re.sub('<.*?>+', ' ', text)
    text = re.sub('_', ' ', text)
    text = re.sub('\n', '', text)
    #text = re.sub('\w*\d\w*', '', text)
    text = re.sub('\'','', text)
    text = re.sub('and', '', text)
    
    
    return text


In [487]:

#Handling stopwords
from collections import Counter
def remove_stopword(text):
    stop_words = stopwords.words('english')
    stopwords_dict = Counter(stop_words)
    text = ' '.join([word for word in text.split() if word not in stopwords_dict])
    return text


In [488]:
import re
#df['description_right'] = df['description_right'].map(lambda x: re.sub(r'\W+', ' ', str(x)))
#df['description_right'] = df['description_right'].replace(r'\W+', ' ', regex=True)

#df['description_right']=df['description_right'].apply(lambda x:mapping_replacer(x, contraction))

#df['description_right']=df['description_right'].apply(lambda x:clean_text(x))




#df['title_right'] = df['title_right'].map(lambda x: re.sub(r'\W+', ' ', str(x)))
#df['title_right'] = df['title_right'].replace(r'\W+', ' ', regex=True)

#df['title_right']=df['title_right'].apply(lambda x:mapping_replacer(x, contraction))

#df['title_right']=df['title_right'].apply(lambda x:clean_text(x))

#df['title_right']=df['title_right'].apply(lambda x:remove_stopword(x))

#df['title_right']=df['title_right'].apply(lambda x: lexicon_normalization(x))

In [489]:

#df['description_left'] = df['description_left'].map(lambda x: re.sub(r'\W+', ' ', str(x)))
#df['description_left'] = df['description_left'].replace(r'\W+', ' ', regex=True)

#df['description_left']=df['description_left'].apply(lambda x: mapping_replacer(x, contraction))

#df['description_left']=df['description_left'].apply(lambda x:clean_text(x))


#df['title_left'] = df['title_left'].map(lambda x: re.sub(r'\W+', ' ', str(x)))
#df['title_left'] = df['title_left'].replace(r'\W+', ' ', regex=True)

#df['title_left']=df['title_left'].apply(lambda x:mapping_replacer(x, contraction))

#df['title_left']=df['title_left'].apply(lambda x:clean_text(x))

#df['title_left']=df['title_left'].apply(lambda x: remove_stopword(x))

#df['title_left']=df['title_left'].apply(lambda x: lexicon_normalization(x))

In [490]:
df['title_right']

0         "Corsair Vengeance LPX CMK64GX4M4A2666C16 - P...
1          "DH0072BALWL HP 72-GB 3G 15K 2.5 DP SAS" "Null"
3         "DISCO DURO SSD Kingston Technology SSDNow V3...
7         "HP Pavilion x360 - 11-ad018ca"@en 11-ad018ca...
8         "StarTech.com 10 100 1000Mbps 32 bit PCI Gigi...
                               ...                        
68432     "TP-Link 8 Port Desktop Switch (TL-SF1008D)"@...
68433     "HDD Desk Black 1TB 3.5 SATA 6Gbs 64MB"@es "T...
68438     "Corsair Carbide Quiet 400Q (CC-9011082-WW) -...
68448      "430165-003 HP 146-GB 3G 10K 2.5 DP SAS" "Null"
68450     "Acer Aspire ES1-132-P194 - Business Notebook...
Name: title_right, Length: 9690, dtype: object

In [491]:
df['title_left']

0         "Corsair Vengeance LPX Black 64GB (4x16GB) DD...
1         "DH0072BALWL HP 72-GB 3G 15K 2.5 DP SAS", "Nu...
3         "DISCO DURO INTERNO SOLIDO HDD SSD KINGSTON V...
7                       "HP Pavilion x360 - 11-ad018ca"@en
8         "Startech 1 Port PCI 10/100/1000 32 Bit Gigab...
                               ...                        
68432     "Switch de Escritorio 8 Puertos 10/100 TP-Lin...
68433     "HDD Desk Black 1TB 3.5 SATA 6Gbs 64MB"@es Tu...
68438     "Carbide 400Q V2 Quiet Desktop Gaming PC Comp...
68448     "430165-003 HP 146-GB 3G 10K 2.5 DP SAS 10 Pa...
68450     "Acer Aspire ES1-132-P194 - Business Notebook...
Name: title_left, Length: 9690, dtype: object

In [492]:
def matching_numbers(title_right, title_left):

    title_right = set(re.findall(r'[0-9]+', title_right))
    title_left = set(re.findall(r'[0-9]+', title_left))    
    union = title_right.union(title_left)
    intersection = title_right.intersection(title_left)

    if len(title_right)==0 and len(title_left) == 0:
        return 1
    else:
        return (len(intersection)/ len(union))


# Implementing Levenshtein Text similarity 

In [493]:

import jellyfish as jf
def engineer_features(df):
    
    df['title_right'] = df['title_right'].str.lower()
    df['title_left'] = df['title_left'].str.lower()

    df['levenshtein_distance'] = df.apply(
    lambda x: jf.levenshtein_distance(x['title_right'], 
                                      x['title_left']), axis=1)

    df['matching_numbers'] = df.apply(
    lambda x: matching_numbers(x['title_right'], 
                               x['title_left']), axis=1)

    df['matching_numbers_log'] = (df['matching_numbers']+1).apply(np.log)
    
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(value=0, inplace=True)

    return df


# All Required Features 

In [494]:
df.head()

,id_left,category_left,id_right,category_right,label,pair_id,description_left,description_right,title_left,title_right,category_match
0,2551242,Computers_and_Accessories,16272671,Computers_and_Accessories,1,2551242#16272671,"""DDR4, 2666MHz, CL16, 1.2v, XMP 2.0, Lifetime ...","""Corsair Vengeance LPX Black 64GB (4x16GB) DD...","""Corsair Vengeance LPX Black 64GB (4x16GB) DD...","""Corsair Vengeance LPX CMK64GX4M4A2666C16 - P...",1
1,16757469,Computers_and_Accessories,16476204,Computers_and_Accessories,1,16757469#16476204,"""Description:2 x 72GB 2.5-inch Serial Attached...","""Description:10 x 72GB 2.5-inch Serial Attach...","""DH0072BALWL HP 72-GB 3G 15K 2.5 DP SAS"", ""Nu...","""DH0072BALWL HP 72-GB 3G 15K 2.5 DP SAS"" ""Null""",1
3,2066119,Computers_and_Accessories,12411100,Computers_and_Accessories,1,2066119#12411100,"""DISCO DURO INTERNO SOLIDO HDD SSD""@es","""DISCO DURO INTERNO SOLIDO HDD SSD KINGSTON V...","""DISCO DURO INTERNO SOLIDO HDD SSD KINGSTON V...","""DISCO DURO SSD Kingston Technology SSDNow V3...",1
7,7759463,Computers_and_Accessories,17348137,Computers_and_Accessories,1,7759463#17348137,up to 2.5 GHz burst frequency connect and sha...,""" Be free to create, connect and share in more...","""HP Pavilion x360 - 11-ad018ca""@en","""HP Pavilion x360 - 11-ad018ca""@en 11-ad018ca...",1
8,9650771,Computers_and_Accessories,11456656,Computers_and_Accessories,1,9650771#11456656,"""Startech 1 Port PCI 10/100/1000 32 Bit Gigab...","""Startech 1 Port PCI 10/100/1000 32 Bit Gigab...","""Startech 1 Port PCI 10/100/1000 32 Bit Gigab...","""StarTech.com 10 100 1000Mbps 32 bit PCI Gigi...",1


In [495]:
df = engineer_features(df)
df = df[['title_left','title_right','levenshtein_distance','matching_numbers','label']]
df

,title_left,title_right,levenshtein_distance,matching_numbers,label
0,"""corsair vengeance lpx black 64gb (4x16gb) dd...","""corsair vengeance lpx cmk64gx4m4a2666c16 - p...",82,0.800000,1
1,"""dh0072balwl hp 72-gb 3g 15k 2.5 dp sas"", ""nu...","""dh0072balwl hp 72-gb 3g 15k 2.5 dp sas"" ""null""",64,1.000000,1
3,"""disco duro interno solido hdd ssd kingston v...","""disco duro ssd kingston technology ssdnow v3...",77,0.333333,1
7,"""hp pavilion x360 - 11-ad018ca""@en","""hp pavilion x360 - 11-ad018ca""@en 11-ad018ca...",31,1.000000,1
8,"""startech 1 port pci 10/100/1000 32 bit gigab...","""startech.com 10 100 1000mbps 32 bit pci gigi...",47,0.800000,1
...,...,...,...,...,...
68432,"""switch de escritorio 8 puertos 10/100 tp-lin...","""tp-link 8 port desktop switch (tl-sf1008d)""@...",63,0.500000,1
68433,"""hdd desk black 1tb 3.5 sata 6gbs 64mb""@es tu...","""hdd desk black 1tb 3.5 sata 6gbs 64mb""@es ""t...",33,1.000000,1
68438,"""carbide 400q v2 quiet desktop gaming pc comp...","""corsair carbide quiet 400q (cc-9011082-ww) -...",90,0.200000,1
68448,"""430165-003 hp 146-gb 3g 10k 2.5 dp sas 10 pa...","""430165-003 hp 146-gb 3g 10k 2.5 dp sas"" ""null""",47,1.000000,1


In [496]:
df=df.sort_values(by=['levenshtein_distance'], ascending=[False])
df

,title_left,title_right,levenshtein_distance,matching_numbers,label
27003,"""hd 2tb sata st2000dm001 seagate""@en ""cftv é ...",""" st1000dm003"", ""st2000dm001""",209,0.400000,1
45051,"""hd 500gb sata st500dm002 seagate""@en ""cftv é...","""seagate desktop hdd 500gb, sata 6gb/s"" 500gb",201,0.333333,1
13195,"""disco duro interno hdd seagate 1tb""","""hd 1tb sata st1000dm003 seagate""@en ""cftv é ...",199,0.333333,1
11494,"""disco duro interno hdd seagate st2000dm001""","""hd 2tb sata st2000dm001 seagate""@en ""cftv é ...",198,0.666667,1
7189,"""hd 2tb sata st2000dm001 seagate""@en ""cftv é ...","""seagate desktop 7200.14 2tb sata3 64mb""@es 6...",196,0.142857,1
...,...,...,...,...,...
8919,"""lenovo thinkserver rd350 - rack-mountable xe...","""lenovo thinkserver rd350 - rack-mountable xe...",1,1.000000,1
61450,"""hyperx fury - ddr3 4 gb dimm 240-pin"" "" hype...","""hyperx fury - ddr3 4 gb dimm 240-pin"" "" hype...",1,1.000000,1
30188,"""amd a8-7650k boxed - prijzen ""@nl tweakers""@nl","""amd a8-7670k boxed - prijzen ""@nl tweakers""@nl",1,0.333333,1
17697,"""tp-link 5-port fast ethernet desktop switch ...","""tp-link 5-port fast ethernet desktop switch ...",0,1.000000,1


# Examining the mean ,count and max values of columns

In [497]:
df.describe()

,levenshtein_distance,matching_numbers,label
count,9690.000000,9690.000000,9690.0
mean,61.339216,0.574844,1.0
std,25.757253,0.302735,0.0
min,0.000000,0.000000,1.0
25%,46.000000,0.333333,1.0
50%,63.000000,0.500000,1.0
75%,78.000000,0.857143,1.0
max,209.000000,1.000000,1.0


In [498]:
df['match'] = np.where(((df['levenshtein_distance']<60) & (df['matching_numbers']>0.6)),1,0)
df['match'].value_counts()

0    7233
1    2457
Name: match, dtype: int64

In [499]:
df

,title_left,title_right,levenshtein_distance,matching_numbers,label,match
27003,"""hd 2tb sata st2000dm001 seagate""@en ""cftv é ...",""" st1000dm003"", ""st2000dm001""",209,0.400000,1,0
45051,"""hd 500gb sata st500dm002 seagate""@en ""cftv é...","""seagate desktop hdd 500gb, sata 6gb/s"" 500gb",201,0.333333,1,0
13195,"""disco duro interno hdd seagate 1tb""","""hd 1tb sata st1000dm003 seagate""@en ""cftv é ...",199,0.333333,1,0
11494,"""disco duro interno hdd seagate st2000dm001""","""hd 2tb sata st2000dm001 seagate""@en ""cftv é ...",198,0.666667,1,0
7189,"""hd 2tb sata st2000dm001 seagate""@en ""cftv é ...","""seagate desktop 7200.14 2tb sata3 64mb""@es 6...",196,0.142857,1,0
...,...,...,...,...,...,...
8919,"""lenovo thinkserver rd350 - rack-mountable xe...","""lenovo thinkserver rd350 - rack-mountable xe...",1,1.000000,1,1
61450,"""hyperx fury - ddr3 4 gb dimm 240-pin"" "" hype...","""hyperx fury - ddr3 4 gb dimm 240-pin"" "" hype...",1,1.000000,1,1
30188,"""amd a8-7650k boxed - prijzen ""@nl tweakers""@nl","""amd a8-7670k boxed - prijzen ""@nl tweakers""@nl",1,0.333333,1,0
17697,"""tp-link 5-port fast ethernet desktop switch ...","""tp-link 5-port fast ethernet desktop switch ...",0,1.000000,1,1


# Applying Validation set For Finding Model Accuracy

In [500]:
data1 = pd.read_json('computers_gs.json',lines = True)
data1.to_csv('test.csv',index = False)
df_test = pd.read_csv('test.csv')
df_test['label'].value_counts()


0    800
1    300
Name: label, dtype: int64

In [501]:
df_test.columns


Index(['id_left', 'category_left', 'cluster_id_left', 'id_right',
       'category_right', 'cluster_id_right', 'label', 'pair_id', 'brand_left',
       'brand_right', 'description_left', 'description_right',
       'keyValuePairs_left', 'keyValuePairs_right', 'price_left',
       'price_right', 'specTableContent_left', 'specTableContent_right',
       'title_left', 'title_right'],
      dtype='object')

In [502]:
df_test.drop(df_test.columns[[2,5,8,9,12,13,14,15,16,17]],axis=1,inplace =True)
df_test


,id_left,category_left,id_right,category_right,label,pair_id,description_left,description_right,title_left,title_right
0,581109,Computers_and_Accessories,16637861,Computers_and_Accessories,0,581109#16637861,"""GV-RX480G1 GAMING-4GD, Core Clock: 1202MHz, B...","""GV-RX550GAMING OC-2GD, Boost: 1219MHz, Memory...","""Gigabyte Radeon RX 480 G1 Gaming 4096MB GDDR...","""Gigabyte Radeon RX 550 Gaming OC 2048MB GDDR..."
1,3083228,Computers_and_Accessories,3424944,Computers_and_Accessories,1,3083228#3424944,"""\n More>>>\n \n...","""\n\n Every det...","""Benq ZOWIE RL2455 24"" Full HD TN Grey comput...","""Zowie RL2455 E-Sports 24"" Full HD LED Monito..."
2,5942105,Computers_and_Accessories,770253,Computers_and_Accessories,0,5942105#770253,"""Apple Mac mini - DTS - 1 x Core i5 2.8 GHz - ...","""Null""@es","""Apple Mac mini 2.8GHz Intel Core i5""@es ""Mac...","""Mac Mini Qc I5 2.6ghz/8gb/1tb/iris Graphics""..."
3,1282014,Computers_and_Accessories,16999524,Other_Electronics,0,1282014#16999524,"""8 port Switch for adding more ports to your r...","""This product is ENERGY STAR qualified for its...","""TP-LINK 8-Port Fast Ethernet Desktop Switch ...","""Tripp Lite 750VA 450W UPS Eco Green Battery ..."
4,7969280,Computers_and_Accessories,6000979,Computers_and_Accessories,0,7969280#6000979,"""A drive USB flash Kingston DataTraveler® 100 ...",NaN,"""Pen Kingston DataTraveler 100 G3 32GB USB3.0...","""Kingston DataTraveler 100 G3 128GB USB3.0""@e..."
...,...,...,...,...,...,...,...,...,...,...
1095,10596400,Computers_and_Accessories,15781433,Computers_and_Accessories,1,10596400#15781433,"""\n More>>>\n \n...","""Description:StorageWorks 20/40GB Internal DL...","""Hewlett Packard Enterprise SP/CQ Drive DLT 2...","""340769-001 HP StorageWorks Internal DLT SCSI..."
1096,10186131,Computers_and_Accessories,14419422,Computers_and_Accessories,1,10186131#14419422,"""1506MHz Core, 1683MHz Boost, 8008MHz 256-bit ...","""ASUS GeForce GTX 1070 ASUS Turbo 8GB GDDR5 VR...","""ASUS GeForce GTX 1070 TURBO 8GB GDDR5 Graphi...","""ASUS NVIDIA GeForce GTX 1070 Turbo 8GB Graph..."
1097,2803663,Video_Games,15959011,Video_Games,1,2803663#15959011,"""DUAL-GTX1060-O6G, Core Clock: 1607MHz, Boost ...","""1594MHz Core, 1809MHz Boost, 8008MHz 192-bit ...","""Asus GeForce GTX 1060 Dual OC 6144MB GDDR5 P...","""ASUS GeForce GTX 1060 DUAL OC 6GB GDDR5 Grap..."
1098,8856662,Computers_and_Accessories,16732972,Computers_and_Accessories,0,8856662#16732972,"""Intuitive closure uses a responsive flex zone...","""The Intel Core i3 processor is the perfect en...","""Speck FlapTop notebook sleeve"" "" Speck sleev...","""Intel Core i3 6300 / 3.8 GHz processor"" "" In..."


In [503]:
df_test['title_right'].isnull().sum()

0

In [504]:
df_test.description_right.fillna(df_test.title_left, inplace = True)
df_test['description_right']

0       "GV-RX550GAMING OC-2GD, Boost: 1219MHz, Memory...
1       "\n\n                                Every det...
2                                              "Null"@es 
3       "This product is ENERGY STAR qualified for its...
4        "Pen Kingston DataTraveler 100 G3 32GB USB3.0...
                              ...                        
1095     "Description:StorageWorks 20/40GB Internal DL...
1096    "ASUS GeForce GTX 1070 ASUS Turbo 8GB GDDR5 VR...
1097    "1594MHz Core, 1809MHz Boost, 8008MHz 192-bit ...
1098    "The Intel Core i3 processor is the perfect en...
1099    "Kingston DataTraveler SE9 - Unidad flash USB ...
Name: description_right, Length: 1100, dtype: object

In [505]:
df_test['description_right'].isnull().sum()

0

In [506]:
df_test['description_left'].isnull().sum()

78

In [507]:
df_test.description_left.fillna(df_test.title_left, inplace = True)
df_test['title_left']

0        "Gigabyte Radeon RX 480 G1 Gaming 4096MB GDDR...
1        "Benq ZOWIE RL2455 24" Full HD TN Grey comput...
2        "Apple Mac mini 2.8GHz Intel Core i5"@es "Mac...
3        "TP-LINK 8-Port Fast Ethernet Desktop Switch ...
4        "Pen Kingston DataTraveler 100 G3 32GB USB3.0...
                              ...                        
1095     "Hewlett Packard Enterprise SP/CQ Drive DLT 2...
1096     "ASUS GeForce GTX 1070 TURBO 8GB GDDR5 Graphi...
1097     "Asus GeForce GTX 1060 Dual OC 6144MB GDDR5 P...
1098     "Speck FlapTop notebook sleeve" " Speck sleev...
1099     "Kingston Technology DataTraveler 100 Generat...
Name: title_left, Length: 1100, dtype: object

In [508]:
df_test['title_left'].isnull().sum()

0

In [509]:
df_test['category_match'] = np.where(df_test['category_left'] == df_test['category_right'],1,0)
df_test

,id_left,category_left,id_right,category_right,label,pair_id,description_left,description_right,title_left,title_right,category_match
0,581109,Computers_and_Accessories,16637861,Computers_and_Accessories,0,581109#16637861,"""GV-RX480G1 GAMING-4GD, Core Clock: 1202MHz, B...","""GV-RX550GAMING OC-2GD, Boost: 1219MHz, Memory...","""Gigabyte Radeon RX 480 G1 Gaming 4096MB GDDR...","""Gigabyte Radeon RX 550 Gaming OC 2048MB GDDR...",1
1,3083228,Computers_and_Accessories,3424944,Computers_and_Accessories,1,3083228#3424944,"""\n More>>>\n \n...","""\n\n Every det...","""Benq ZOWIE RL2455 24"" Full HD TN Grey comput...","""Zowie RL2455 E-Sports 24"" Full HD LED Monito...",1
2,5942105,Computers_and_Accessories,770253,Computers_and_Accessories,0,5942105#770253,"""Apple Mac mini - DTS - 1 x Core i5 2.8 GHz - ...","""Null""@es","""Apple Mac mini 2.8GHz Intel Core i5""@es ""Mac...","""Mac Mini Qc I5 2.6ghz/8gb/1tb/iris Graphics""...",1
3,1282014,Computers_and_Accessories,16999524,Other_Electronics,0,1282014#16999524,"""8 port Switch for adding more ports to your r...","""This product is ENERGY STAR qualified for its...","""TP-LINK 8-Port Fast Ethernet Desktop Switch ...","""Tripp Lite 750VA 450W UPS Eco Green Battery ...",0
4,7969280,Computers_and_Accessories,6000979,Computers_and_Accessories,0,7969280#6000979,"""A drive USB flash Kingston DataTraveler® 100 ...","""Pen Kingston DataTraveler 100 G3 32GB USB3.0...","""Pen Kingston DataTraveler 100 G3 32GB USB3.0...","""Kingston DataTraveler 100 G3 128GB USB3.0""@e...",1
...,...,...,...,...,...,...,...,...,...,...,...
1095,10596400,Computers_and_Accessories,15781433,Computers_and_Accessories,1,10596400#15781433,"""\n More>>>\n \n...","""Description:StorageWorks 20/40GB Internal DL...","""Hewlett Packard Enterprise SP/CQ Drive DLT 2...","""340769-001 HP StorageWorks Internal DLT SCSI...",1
1096,10186131,Computers_and_Accessories,14419422,Computers_and_Accessories,1,10186131#14419422,"""1506MHz Core, 1683MHz Boost, 8008MHz 256-bit ...","""ASUS GeForce GTX 1070 ASUS Turbo 8GB GDDR5 VR...","""ASUS GeForce GTX 1070 TURBO 8GB GDDR5 Graphi...","""ASUS NVIDIA GeForce GTX 1070 Turbo 8GB Graph...",1
1097,2803663,Video_Games,15959011,Video_Games,1,2803663#15959011,"""DUAL-GTX1060-O6G, Core Clock: 1607MHz, Boost ...","""1594MHz Core, 1809MHz Boost, 8008MHz 192-bit ...","""Asus GeForce GTX 1060 Dual OC 6144MB GDDR5 P...","""ASUS GeForce GTX 1060 DUAL OC 6GB GDDR5 Grap...",1
1098,8856662,Computers_and_Accessories,16732972,Computers_and_Accessories,0,8856662#16732972,"""Intuitive closure uses a responsive flex zone...","""The Intel Core i3 processor is the perfect en...","""Speck FlapTop notebook sleeve"" "" Speck sleev...","""Intel Core i3 6300 / 3.8 GHz processor"" "" In...",1


In [510]:
df_test['category_match'].value_counts()

1    1063
0      37
Name: category_match, dtype: int64

In [511]:
df_test.drop(df_test.index[df_test['category_match'] == 0],inplace = True)

In [512]:
df_test['category_match'].value_counts()

1    1063
Name: category_match, dtype: int64

In [513]:
df_test = engineer_features(df_test)
df_test = df_test[['title_left','title_right','levenshtein_distance','matching_numbers','label']]
df_test

,title_left,title_right,levenshtein_distance,matching_numbers,label
0,"""gigabyte radeon rx 480 g1 gaming 4096mb gddr...","""gigabyte radeon rx 550 gaming oc 2048mb gddr...",11,0.166667,0
1,"""benq zowie rl2455 24"" full hd tn grey comput...","""zowie rl2455 e-sports 24"" full hd led monito...",89,0.666667,1
2,"""apple mac mini 2.8ghz intel core i5""@es ""mac...","""mac mini qc i5 2.6ghz/8gb/1tb/iris graphics""...",73,0.750000,0
4,"""pen kingston datatraveler 100 g3 32gb usb3.0...","""kingston datatraveler 100 g3 128gb usb3.0""@e...",41,0.600000,0
5,"""fire, 7"" display, wi-fi, 8 gb - includes spe...","""apple ipad mini 4 wi-fi - tablet 128 gb 7.9""...",118,0.111111,0
...,...,...,...,...,...
1095,"""hewlett packard enterprise sp/cq drive dlt 2...","""340769-001 hp storageworks internal dlt scsi...",101,0.500000,1
1096,"""asus geforce gtx 1070 turbo 8gb gddr5 graphi...","""asus nvidia geforce gtx 1070 turbo 8gb graph...",36,0.500000,1
1097,"""asus geforce gtx 1060 dual oc 6144mb gddr5 p...","""asus geforce gtx 1060 dual oc 6gb gddr5 grap...",41,0.500000,1
1098,"""speck flaptop notebook sleeve"" "" speck sleev...","""intel core i3 6300 / 3.8 ghz processor"" "" in...",81,0.000000,0


In [515]:
X_train=df[['levenshtein_distance','matching_numbers']]
X_test=df_test[['levenshtein_distance','matching_numbers']]
y_train=df['match']
y_test=df_test['label']
y_train

array([0, 0, 0, ..., 0, 1, 0])

# Creating Function For Finding Confusion Matrix

In [516]:
def get_confusion_matrix_values(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    return(cm[0][0], cm[0][1], cm[1][0], cm[1][1])

# Model Building: Decision Tree,Random Forest and Support Vector Classifier

In [517]:
from sklearn.svm import SVC
classifiers = {
    "DecisionTreeClassifier":DecisionTreeClassifier(max_depth=2),
    "Support Vector Classifier":SVC(),
    "RandomForestClassifier":RandomForestClassifier(n_estimators=1000,max_depth=2),

}


df_results = pd.DataFrame(columns=['model', 'accuracy', 'precision',
                                   'true_pos','false_pos',
                                   'true_neg','false_neg','recall','f1'])

for key in classifiers:

    classifier = classifiers[key]
    model = classifier.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, zero_division=0)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    classification = classification_report(y_test, y_pred, zero_division=0)
    tp, fp, fn, tn = get_confusion_matrix_values(y_test, y_pred)

    row = {'model': key,
           'accuracy': accuracy,
           'precision': precision,
           'Recall': recall,
           'f1': f1,
           'true_pos': tp,
           'false_pos': fp,
           'true_neg': tn,
           'false_neg': fn,
          }
    df_results = df_results.append(row, ignore_index=True)

df_results.head(10)

,model,accuracy,precision,true_pos,false_pos,true_neg,false_neg,recall,f1,Recall
0,DecisionTreeClassifier,1.0,1.0,2170,0,737,0,NaN,1.0,1.000000
1,Support Vector Classifier,0.845201,0.654467,1849,321,608,129,NaN,0.729892,0.824966
2,RandomForestClassifier,1.0,1.0,2170,0,737,0,NaN,1.0,1.000000


# Estimating Results in Binary

In [518]:
results = pd.DataFrame(data={'predictions': y_pred, 'actual': y_test})
results['result'] = np.where(results['predictions']==results['actual'], 1, 0)
results


,predictions,actual,result
0,1,1,1
1,1,1,1
2,0,0,1
3,0,0,1
4,0,0,1
...,...,...,...
2902,0,0,1
2903,0,0,1
2904,0,0,1
2905,0,0,1


# Final Results with Match and Not Matched Classification

In [519]:
results['predictions'].replace(0, 'Not match',inplace=True)
results['predictions'].replace(1, 'Match',inplace=True)

results['actual'].replace(0, 'Not Match',inplace=True)
results['actual'].replace(1, 'Match',inplace=True)

results['result'].replace(0, 'False',inplace=True)
results['result'].replace(1, 'True',inplace=True)
results

,predictions,actual,result
0,Match,Match,True
1,Match,Match,True
2,Not match,Not Match,True
3,Not match,Not Match,True
4,Not match,Not Match,True
...,...,...,...
2902,Not match,Not Match,True
2903,Not match,Not Match,True
2904,Not match,Not Match,True
2905,Not match,Not Match,True
